# DATA 558 Midterm

Will Wright

### Exercise 1

**Instructions**  
Compute the gradient $\nabla F(\beta)$ where the objective is:
$$\min_{\mathbf{\beta \in \mathbb{R}^d}} F(\beta):=\frac{1}{n}\sum_{i=1}^{n} \frac{1}{\rho}log(1+exp(-\rho y_ix_i^T\beta)) + \lambda\lVert\beta\rVert_2^2$$

**Solution**  
Start by moving the scalar $\frac{1}{\rho}$ outside the summation:  
$F(\beta)=\frac{1}{n\rho}\sum_{i=1}^{n} log(1+exp(-\rho y_ix_i^T\beta)) + \lambda\lVert\beta\rVert_2^2$

Next, break up into two terms:  
(1) $\frac{1}{n\rho}\sum_{i=1}^{n} log(1+exp(-\rho y_ix_i^T\beta))$  
(2) $\lambda\lVert \beta\rVert_2^2$ 

Find the derivative of the first term:  
(1) $\frac{\partial}{\partial \beta}\Big[\frac{1}{n\rho}\sum_{i=1}^{n} log(1+exp(-\rho y_ix_i^T\beta))\Big]$  
  
> Move the constant and summation to the outside of the derivative: 
$\frac{1}{n\rho}\sum_{i=1}^{n}\frac{\partial}{\partial \beta}\Big[ log(1+exp(-\rho y_ix_i^T\beta))\Big]$ 
  
> Use the chain rule with the following functions and their derivatives:  
$(f\circ g \circ h) = log(g\circ h)$  
$(f\circ g \circ h)' = \frac{1}{(g\circ h)} \cdot (g \circ h)' $  
$(g\circ h) = 1+exp(-h)$  
$(g\circ h)' = -exp(-h)\cdot h'$  
$h = \rho y_ix_i^T\beta$  
$h' = \rho y_ix_i$  
  
> Putting the chains together, we have:  
$(f\circ g \circ h)' = -\rho y_ix_i \cdot \frac{exp(- \rho y_ix_i^T\beta)}{1+exp(-\rho y_ix_i^T\beta)}$
  
> Re-apply constant scalar and summation:  
$=\frac{1}{n\rho}\sum_{i=1}^{n}-\rho y_ix_i \cdot \frac{exp(- \rho y_ix_i^T\beta)}{1+exp(-\rho y_ix_i^T\beta)}$

> Move the constant $-\rho$ outside the summation and cancel it out:
$=-\frac{1}{n}\sum_{i=1}^{n}y_ix_i \cdot \frac{exp(- \rho y_ix_i^T\beta)}{1+exp(-\rho y_ix_i^T\beta)}$

Next, derive the second term:  
(2) $\frac{\partial}{\partial \beta}\lambda\lVert \beta\rVert_2^2$  
  
> Move the constant $\lambda$ outside the derivation and convert to matrix form:  
$ =\lambda \frac{\partial}{\partial \beta}\beta^T \beta$
  
> Multiply by the identity matrix $I$:  
$ =\lambda \frac{\partial}{\partial \beta}\beta^T I \beta$
  
> Apply the property $\frac{\partial}{\partial x}x^T Ax = (A+A^T)x$:  
$ =\lambda (I + I^T)\beta$  
  
> Given that $(I + I^T)$ is simply a scalar of 2 for matrices, we can simplify:  

> $=2\lambda \beta $  
  
Next, we add (1) and (2) to get:  
$\nabla F(\beta) = -\frac{1}{n}\sum_{i=1}^{n}y_ix_i \cdot \frac{exp(- \rho y_ix_i^T\beta)}{1+exp(-\rho y_ix_i^T\beta)} + 2\lambda \beta$

**Instructions**  
Consider the Spam dataset from The Elements of Statistical Learning. Standardize the data, if you have not done so already. Be sure to use the training and test splits from the website.

In [117]:
# Load Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.preprocessing
import scipy.linalg
from sklearn.linear_model import LogisticRegression

spam = pd.read_table('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/spam.data', 
                   delim_whitespace=True, header = None)
test_indicator = pd.read_table('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/spam.traintest',
                         delim_whitespace=True, header = None)

In [100]:
x = np.asarray(spam)[:, 0:-1]
y = np.asarray(spam)[:, -1]*2 - 1
test_indicator = np.array(test_indicator).T[0]

# Divide the data into train, test sets
x_train = x[test_indicator == 0, :]
x_test = x[test_indicator == 1, :]
y_train = y[test_indicator == 0]
y_test = y[test_indicator == 1]

# Standardize the data.
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Keep track of the number of samples and dimension of each sample
n_train = len(y_train)
n_test = len(y_test)
d = np.size(x, 1)

**Instructions**  
Write a function _myrhologistic_ that implements the accelerated gradient algorithm to train the $\ell_2^2$-regularized binary logistic regression with $\rho$-logistic loss. The function takes as input
the initial step-size for the backtracking rule, the $\epsilon$ for the stopping criterion based on the norm of the gradient of the objective, and the value of $\rho$.

In [243]:
# Subroutines for myrhologistic to call:

# rho logistic gradient
def computegrad(beta, lamb, rho, x, y):
    yx = y[:, np.newaxis]*x
    denom = 1+np.exp(-rho*yx.dot(beta))
    grad = -1/len(y)*np.sum(yx*np.exp(-rho*yx.dot(beta[:, np.newaxis]))/
        denom[:, np.newaxis], axis=0) + 2*lamb*beta
    return grad

# rho logistic objective
def computeobj(beta, lamb, rho, x, y):
    return 1/len(y) * np.sum(1/rho*np.log(1 + np.exp(-rho*y*x.dot(beta)))) + lamb*np.linalg.norm(beta)**2

# backtracking with rho logistic
def backtracking(beta, lamb, rho, x, y, eta, alpha=0.5, gamma=0.8):
    grad_beta = computegrad(beta, lamb, rho, x, y)
    norm_grad_beta = np.linalg.norm(grad_beta)
    found_eta = False
    iter = 0
    while found_eta == False:
        if computeobj(beta - eta * grad_beta, lamb, rho, x, y) < \
            computeobj(beta, lamb, rho, x, y)- alpha * eta * norm_grad_beta ** 2:
                found_eta = True
        else:
            eta *= gamma
            iter += 1
    return eta

In [259]:
def computeEtaInit(x, y, lamb):
    eta_init = 1/(scipy.linalg.eigh(1/len(y)*x.T.dot(x),
                                eigvals=(d-1, d-1),
                                eigvals_only=True)[0]+lamb)
    return eta_init

In [309]:
def myrhologistic(x, y, beta_init, theta_init, lamduh, rho, eta_init, eps):
    beta = beta_init
    theta = theta_init
    eta = eta_init
    grad_theta = computegrad(theta, lamb, rho, x, y)
    grad_beta = computegrad(beta, lamb, rho, x, y)
    beta_vals = beta
    theta_vals = theta
    iter = 0
    while np.linalg.norm(grad_beta) > eps:
        eta = backtracking(theta, lamb, rho, x, y, eta)
        beta_new = theta - eta*grad_theta
        theta = beta_new + iter/(iter+3)*(beta_new-beta)
        beta_vals = np.vstack((beta_vals, beta))
        theta_vals = np.vstack((theta_vals, theta))
        grad_theta = computegrad(theta, lamb, rho, x, y)
        grad_beta = computegrad(beta, lamb, rho, x, y)
        beta = beta_new
        iter += 1
    return beta_vals

**Instructions**  
Train your $\ell_2^2$-regularized binary logistic regression with $\rho$-logistic loss with $\rho=2$ and $\epsilon=10^{-3}$ o the Spam dataset for $\lambda=1$. Report your misclassification error for this value of $\lambda$.

In [261]:
# initialize parameters
rho = 2
eps = 10**-3
lamb = 1
beta_init = np.zeros(d)
theta_init = np.zeros(d)
eta_init = computeEtaInit(x_train, y_train, lamb)

In [310]:
train_betas = myrhologistic(x_train, y_train, beta_init, theta_init, lamb, rho, eta_init, eps)

In [311]:
beta_opt = train_betas[-1,:]

In [312]:
def compute_error(x, y, beta):
    y_pred = 1/(1+np.exp(-x.dot(beta))) > 0.5
    y_pred = y_pred*2 - 1 # Convert to +/- 1
    return np.mean(y_pred != y)

In [313]:
compute_error(x_train, y_train, beta_opt)

0.09526916802610114

In [ ]:
def plot_misclassification_error(betas_grad, betas_fastgrad, x, y, save_file='', title=''):
    niter_grad = np.size(betas_grad, 0)
    error_grad = np.zeros(niter_grad)
    niter_fg = np.size(betas_fastgrad, 0)
    error_fastgrad = np.zeros(niter_fg)
    for i in range(niter_grad):
        error_grad[i] = compute_misclassification_error(betas_grad[i, :], x, y)
    for i in range(niter_fg):
        error_fastgrad[i] = compute_misclassification_error(betas_fastgrad[i, :], x, y)
    fig, ax = plt.subplots()
    ax.plot(range(1, niter_grad + 1), error_grad, label='gradient descent')
    ax.plot(range(1, niter_fg + 1), error_fastgrad, c='red', label='fast gradient')
    plt.xlabel('Iteration')
    plt.ylabel('Misclassification error')
    if title:
        plt.title(title)
    ax.legend(loc='upper right')
    if not save_file:
        plt.show()
    else:
        plt.savefig(save_file)

**Instructions**  
Write a function _crossval_ that implements leave-one-out cross-validation and hold-out cross-validation. You may either write a function that implements each variant separately depending on the case, or write a general cross-validation function that can be instantiated in each case.

In [347]:
def loocv(x, y, lamb, rho, eps):
    n, d = x.shape
    beta_init = np.zeros(d)
    theta_init = np.zeros(d)
    errors = []
    for i in range(n):
        x_loo = np.vstack((x[0:i,:],x[(i+1):,:]))
        y_loo = np.concatenate([y[0:i],y[(i+1):]])
        eta_init = computeEtaInit(x_loo, y_loo, lamb)
        loo_betas = myrhologistic(x_loo, y_loo, beta_init, theta_init, lamb, rho, eta_init, eps)
        loo_beta_opt = loo_betas[-1,:]
        errors.append(compute_error(x_train, y_train, beta_opt))
    return np.sum(errors)/n
    

In [348]:
test = loocv(x_train, y_train, lamb, rho, eps)

In [349]:
test

0.09526916802610112

In [353]:
def hocv(x, y, lamb, eps, rho, test_size = 0.2):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    d = x_train.shape[1]
    beta_init = np.zeros(d)
    theta_init = np.zeros(d)
    eta_init = computeEtaInit(x_train, y_train, lamb)
    ho_betas = myrhologistic(x_train, y_train, beta_init, theta_init, lamb, rho, eta_init, eps)
    ho_beta_opt = ho_betas[-1,:]
    error = compute_error(x_test, y_test, beta_opt)
    return error

In [356]:
hocv(x_train, y_train, lamb, eps, rho, test_size = 0.2)

0.11745513866231648

**Instructions**  
Find the optimal value of $\lambda$ using leave-one-out cross-validation.  Find the optimal value of $\lambda$ useing hold-out cross-validation with a 80%/20% split for the training set/testing set.  Report your misclassification errors for the two values of $\lambda$ found.

In [417]:
lamb_vals = []
for i in range(-3,3):
    lamb_vals.append(10**i) # testing very small to very large
# lamb_vals.extend(np.linspace(0,2,21)) # after seeing 1 be optimal, testing values around 1

In [371]:
def compute_lamb_opt(x, y, lamb_vals, eps, rho, method, test_size = 0.2):
    if method=='loocv':
        errors = []
        for lamb in lamb_vals:
            error = loocv(x, y, lamb, rho, eps)
            errors.append(error)
        minpos = errors.index(min(errors))
        lamb_opt = lamb_vals[minpos]
        return lamb_opt
    if method=='hocv':
        errors = []
        for lamb in lamb_vals:
            error = hocv(x, y, lamb, eps, rho, test_size)
            errors.append(error)
        minpos = errors.index(min(errors))
        lamb_opt = lamb_vals[minpos]
        return lamb_opt
    else:
        print('Unknown method')

In [419]:
loocv_lamb_opt = compute_lamb_opt(x_train, y_train, lamb_vals, eps, rho, method = 'loocv')

In [420]:
loocv_lamb_opt

0.001

In [421]:
loocv(x_train, y_train, loocv_lamb_opt, rho, eps)

0.09526916802610112

In [428]:
hocv_lamb_opt = compute_lamb_opt(x_train, y_train, lamb_vals, eps, rho, method = 'hocv')

In [429]:
hocv_lamb_opt

0.1

In [430]:
hocv(x_train, y_train, hocv_lamb_opt, eps, rho, test_size = 0.2)

0.09624796084828711

# Exercise 2 - Data Competition Project

**Instructions**  
Pick two classes of your choice from the dataset. Train a classifier using $\ell_2^2$-regularized binary
logistic regression with $\rho$-logistic loss on the training set using your own accelerated gradient algorithm with $\rho = 2$, $\epsilon = 10^{−3}$, and $\lambda = 1$. Be sure to use the features you previously generated with the provided script rather than the raw image features. Plot, with different colors, the misclassification error on the training set and on the validation set vs iterations.

**Instructions**  
Find the value of the regularization parameter $\lambda$ using using leave-one-out cross-validation. Find the value of the regularization parameter $\lambda$ using using hold-out cross-validation. Train a classifier using $\ell_2^2$-regularized binary logistic regression with $\rho$-logistic loss on the training set using your own accelerated gradient algorithm with that value of $\lambda$ found by hold-out cross-validation. Plot, with different colors, the misclassification error on the training set and on the validation set vs. iterations.

**Instructions**  
Consider all pairs of classes from the dataset. For each pair of classes, train a classifier using
a $\ell_2^2$-regularized binary logistic regression with $\rho$-logistic loss on the training set comprising
only the data-points for that pair of classes using your own fast gradient algorithm. For each
pair of classes, find the value of the regularization parameter $\lambda$ using hold-out cross-validation
on the training set comprising only the data-points for that pair of classes.

**Instructions**  
Write a function that for any new data point predicts its label. To do this, you will perform the following: input the data point into each classifier (for each pair of classes) you trained above. Record the class predicted by each classifier. Then your prediction for this data point is the most frequently predicted class. If there is a tie, randomly choose between the tied classes. Report the misclassification error on the validation set and test set. Report the precision/recall on the validation set.
